In [ ]:
from collections import defaultdict
from os.path import join, exists, splitext, basename, dirname, isdir
from os import listdir, symlink, makedirs
from shutil import copyfile
from praatio import tgio
from tqdm import tqdm
from glob import glob
import librosa
import pandas as pd
import scipy.io.wavfile as wav

from cac.utils.io import load_json, write_txt
from cac.utils.pandas import apply_antifilters

In [ ]:
# directory where the data resides
data_root = '/data/flusense/'

In [ ]:
# src and destination directories
load_dir = join(data_root, 'raw')
save_root = join(data_root, 'processed')

In [ ]:
makedirs(save_root, exist_ok=True)

In [ ]:
load_audio_dir = join(load_dir, 'audio')
load_annotation_dir = join(load_dir, 'annotations')

In [ ]:
save_audio_dir = join(save_root, 'audio')
makedirs(save_audio_dir, exist_ok=True)

#### Important check: find out files that are unreadable via `EOFError` (can't be discovered by `librosa`)

In [ ]:
files = listdir(load_audio_dir)

In [ ]:
len(files)

In [ ]:
files[0]

In [ ]:
invalid_files = []

for file in tqdm(files, desc='Checking valid files'):
    fpath = f'/data/flusense/raw/audio/{file}'
    try:
        fs,signal = wav.read(fpath)
    except:
        invalid_files.append(file)

In [ ]:
len(invalid_files)

In [ ]:
# files = list(set(files) - set(invalid_files))

In [ ]:
# len(files)

### Create symlinks to the original .wav files

In [ ]:
len(files)

In [ ]:
def check_exists(fname):
    fpath = join("/data/flusense/FluSense-data/FluSense-audio/", fname)
    return exists(fpath)

In [ ]:
check_exists(files[0])

In [ ]:
files = [f for f in files if check_exists(f)]

In [ ]:
len(files)

In [ ]:
for file in tqdm(files, desc='Creating symlinks processed/ <- raw/'):
    dest = join(save_audio_dir, file)
    if not exists(dest):
        symlink(join(load_audio_dir, file), dest)

### Creating symlinks for annotations

Annotation files are .TextGrid => using library praatio to read them

Steps for each file:

* Checked that each annotation object satisfies len(annotation.tierNameList) == 1
* Extract the list of entries
* For each entry, add the label of that entry to the list of classification labels for that file and add each interval to the list of intervals for that file

Final goal is to obtain classification_targets and segmentation_targets for all the files

In [ ]:
classification_targets = []
segmentation_targets = []

for file in files:
    _classification_targets = set()
    _segmentation_targets = defaultdict(list)

    text_grid = tgio.openTextgrid(join(load_annotation_dir, file.replace('wav', 'TextGrid')))
    
    # ensure that only one name in the namelist
    assert len(text_grid.tierNameList) == 1
    
    t_name = text_grid.tierNameList[0]
    
    # this is a list of entries
    # each entry consists of an interval
    entry_list = text_grid.tierDict[t_name].entryList
    
    for entry in entry_list:
        label = entry.label
        start = entry.start
        end = entry.end
        
        # add label to classification targets for that file
        _classification_targets.add(label)
        # add interval to segmentation targets for that file
        _segmentation_targets[label].append([start, end])
    
    classification_targets.append(list(_classification_targets))
    segmentation_targets.append(dict(_segmentation_targets))

In [ ]:
starts = [0.0 for _ in files]
ends = [librosa.get_duration(filename=join(save_audio_dir, x)) for x in tqdm(files)]

In [ ]:
# remove .wav from the filenames
files = [splitext(file)[0] for file in files]

In [ ]:
# create dataframe storing the data
df = pd.DataFrame({'file': files, 'classification': classification_targets, 'segmentation': segmentation_targets, 'start': starts, 'end': ends})

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
invalid_files

In [ ]:
# drop invalid/unreadable files
df = apply_antifilters(df, {'file': [x.split('.wav')[0] for x in invalid_files]})

In [ ]:
df.shape

In [ ]:
# save the dataframe
annotation_save_path = join(save_root, 'annotation.csv')
df.to_csv(annotation_save_path, index=False)

In [ ]:
description = "Annotation columns: \n \
`classification`: valid labels = [cough, sneeze, sniffle, throat-clearing,\
 speech, etc (i.e everything else)] \n \
`segmentation`: {label: list of lists, each marking the start and end of the\
 interval in which the label is occuring in the file}"

with open(join(save_root, 'description.txt'), 'w') as f:
    f.write(description)